In [1]:
CHOROPLETH_DATA_FILE = '../data/processed/choropleth_df.csv'

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [4]:
%load_ext autoreload
%autoreload 2

## Read in data

In [61]:
df = pd.read_csv(CHOROPLETH_DATA_FILE)
df.head()

## Process data
Here we'll make some small changes to the data and perform some checks, in order to create a nice world map with suicide rates

In [63]:
both = df.groupby(['year', 'country']).agg(population = pd.NamedAgg(column="population", aggfunc=sum), 
                                                 suicides_no = pd.NamedAgg(column="suicides_no", aggfunc=sum), 
                                                 code = pd.NamedAgg(column="code", aggfunc=lambda x: x[0]),
                                                ).reset_index()
females = df[df["sex"] == "female"]
males = df[df["sex"] == "male"]
both["suicides per 100,000"] = both['suicides_no'] / both['population'] * 100000 

In [64]:
# some checks on whether everything is still ok 
fem_pop_arg = females.loc[(females["country"] == "Argentina") & (females["year"] == 1979), "population"].values
ma_pop_arg = males.loc[(males["country"] == "Argentina") & (males["year"] == 1979), "population"].values
both_pop_arg = both.loc[(both["country"] == "Argentina") & (both["year"] == 1979), "population"].values
assert both_pop_arg == fem_pop_arg + ma_pop_arg

In [66]:
both.head()

,year,country,population,suicides_no,code,"suicides per 100,000"
0,1979,Argentina,25000000.0,1802.0,ARG,7.208000
1,1979,Australia,13280000.0,1675.0,AUS,12.612952
2,1979,Austria,7060000.0,0.0,AUT,0.000000
3,1979,Bahamas,189000.0,0.0,BHS,0.000000
4,1979,Barbados,230000.0,2.0,BRB,0.869565


In [68]:
males.head()

,year,sex,country,suicides_no,population,"suicides per 100,000",code
56,1979,male,Argentina,1294.0,12500000.0,10.39,ARG
57,1979,male,Australia,1196.0,6640000.0,18.02,AUS
58,1979,male,Austria,0.0,3320000.0,0.00,AUT
59,1979,male,Bahamas,0.0,93200.0,0.00,BHS
60,1979,male,Barbados,2.0,110000.0,1.83,BRB


# Create choropleth map

Here we're creating a graph which shows a world map coloured by suicide rate per 100,000 people. I created one for males, females, and an overall combined plot

In [69]:
px.choropleth(both, locations="code", color="suicides per 100,000", hover_name="country", animation_frame="year",
             color_continuous_scale="Reds", 
             hover_data=["population"])

In [71]:
px.choropleth(females, locations="code", color="suicides per 100,000", hover_name="country", animation_frame="year",
             color_continuous_scale="Reds", 
             hover_data=["population"])

In [75]:
px.choropleth(males, locations="code", color="suicides per 100,000", hover_name="country", animation_frame="year",
             color_continuous_scale="Reds", 
             hover_data=["population"])